
# **Метод Ньютона и модификация Федоренко для решения системы нелинейных уравнений**

## Постановка задачи

Рассматриваем систему нелинейных уравнений:

$$
% begin{cases}
f_1(x, y) = x^5 + y^4 - 2 = 0, \
f_2(x, y) = (x - 2)^3 + (y - 2)^3 + 16 = 0.
% \end{cases}
$$

Цель — найти приближение решения $(x^*, y^*)$, такого что $F(x^*, y^*) = 0$.

Начальное приближение:

$
(x_0, y_0) = (2, 3)
$

---

##  Теория метода Ньютона для систем

Пусть дана система:

$$
F(x) = 0,\qquad
F:\mathbb{R}^n \to \mathbb{R}^n.
$$

Метод Ньютона строит последовательность:

$$
x_{k+1} = x_k + \Delta x_k.
$$

где приращение (\Delta x_k) находится из **линейной аппроксимации (Тейлор 1-го порядка)**:

$$
F(x_k + \Delta x) \approx F(x_k) + J(x_k)\Delta x.
$$

Требуем, чтобы новое приближение лучше удовлетворяло системе:

$$
F(x_k + \Delta x) \approx 0.
$$

Получаем линейную систему:

$$
J(x_k)\Delta x_k = -F(x_k),
$$

где
$J(x)$ — матрица Якоби:

$$
J(x,y)=
\begin{pmatrix}
\frac{\partial f_1}{\partial x} & \frac{\partial f_1}{\partial y} \
\frac{\partial f_2}{\partial x} & \frac{\partial f_2}{\partial y}
\end{pmatrix}.
$$



##  Решение системы 2×2: формулы Крамера

Для матрицы

$$
J=
\begin{pmatrix}
a & b \
c & d
\end{pmatrix}, \qquad
F=
\begin{pmatrix}
f_1\
f_2
\end{pmatrix}
$$

система:

$$
J\Delta = -F
$$

имеет решение:

$$
\Delta x_1 = \frac{(-f_1)d - b(-f_2)}{ad - bc},\quad
\Delta x_2 = \frac{a(-f_2) - (-f_1)c}{ad - bc}.
$$

Детерминант:

$$
\det(J)=ad - bc.
$$


## Метод Ньютона–Федоренко

Федоренко предложил модификацию, включающую:

---

##  Масштабирование уравнений

Вес каждого уравнения определяется как длина его градиента:

$$
\alpha_i = |\nabla f_i(x)|
$$

Используется **взвешенная норма**:

$$
|F(x)|_\alpha
= \sqrt{
\left(\frac{f_1}{\alpha_1}\right)^2 +
\left(\frac{f_2}{\alpha_2}\right)^2
}.
$$



## Подбор шага τ 

В классическом методе Ньютона:

$$
\tau = 1.
$$

У Федоренко:

1. Пробуем полный шаг.
2. Если норма **не уменьшилась**, уменьшаем τ:
   $$
   \tau \leftarrow \frac{\tau}{2}.
   $$
3. Повторяем, пока улучшение не наступит.


In [6]:
import math

eps = 1e-15
maxIter = 50


def systemEquations(x, y):
    """
    Система уравнений:
    f1 = x^5 + y^4 - 2
    f2 = (x-2)^3 + (y-2)^3 + 16
    """
    f1 = x**5 + y**4 - 2.0
    f2 = (x - 2)**3 + (y - 2)**3 + 16.0
    return [f1, f2]


def systemJacobian(x, y):
    """
    Матрица Якоби:
    df1/dx = 5x^4
    df1/dy = 4y^3
    df2/dx = 3(x-2)^2
    df2/dy = 3(y-2)^2
    """
    j00 = 5 * x**4
    j01 = 4 * y**3
    j10 = 3 * (x - 2)**2
    j11 = 3 * (y - 2)**2
    return [j00, j01, j10, j11]


def computeAlphaCoefficients(j):
    """
    Весовые коэффициенты Федоренко
    """
    normF1 = math.sqrt(j[0]**2 + j[1]**2)
    normF2 = math.sqrt(j[2]**2 + j[3]**2)
    return [normF1, normF2]


def computeNorm(f, alpha):
    """
    Вычисление взвешенной нормы
    """
    term1 = (f[0] / alpha[0])**2 if alpha[0] != 0 else 0
    term2 = (f[1] / alpha[1])**2 if alpha[1] != 0 else 0
    return math.sqrt(term1 + term2)


def solve2x2System(j, f):
    """
    Решение системы J * dx = -F формулами Крамера
    """
    a, b, c, d = j
    det = a * d - b * c

    if abs(det) < 1e-30:
        raise ValueError("Якобиан вырожден")

    dx1 = ( (-f[0]) * d - b * (-f[1]) ) / det
    dx2 = ( a * (-f[1]) - (-f[0]) * c ) / det
    return [dx1, dx2]


def fedorenkoMethod(x0, y0, eps=eps, maxIter=maxIter, verbose=False):
    """
    Метод Ньютона с модификацией Федоренко
    """
    x, y = x0, y0
    oldNorm = 1e300

    for iteration in range(maxIter):
        f = systemEquations(x, y)
        j = systemJacobian(x, y)

        alpha = computeAlphaCoefficients(j)
        currentNorm = computeNorm(f, alpha)

        if verbose:
            print(f"iter {iteration:2d}: x={x:.6f}, y={y:.6f}, |F|={currentNorm:.3e}")

        if currentNorm < eps:
            return x, y, iteration, currentNorm

        dx = solve2x2System(j, f)

        tau = 1.0
        while tau > 1e-15:
            xNew = x + tau * dx[0]
            yNew = y + tau * dx[1]

            fNew = systemEquations(xNew, yNew)
            newNorm = computeNorm(fNew, alpha)

            if newNorm < currentNorm:
                x, y = xNew, yNew
                break

            tau /= 2

        if tau <= 1e-15:
            if verbose:
                print("Шаг не удалось уменьшить — остановка")
            break

    return x, y, maxIter, -1


result = fedorenkoMethod(2.0, 3.0, verbose=True)
x, y, iters, err = result
print(f"x={x:.6f}, y={y:.6f}, итераций={iters}, ошибка={err}")


iter  0: x=2.000000, y=3.000000, |F|=5.727e+00
iter  1: x=2.782812, y=2.291667, |F|=8.915e+00
iter  2: x=2.779014, y=2.315083, |F|=8.969e+00
iter  3: x=2.778542, y=2.317914, |F|=8.975e+00
iter  4: x=2.778439, y=2.318530, |F|=8.977e+00
iter  5: x=3.119913, y=0.277460, |F|=1.415e+00
iter  6: x=2.500342, y=-0.841726, |F|=5.672e-01
iter  7: x=2.011562, y=-0.544943, |F|=3.798e-01
iter  8: x=1.632799, y=-0.520081, |F|=2.723e-01
iter  9: x=1.360594, y=-0.511464, |F|=1.594e-01
iter 10: x=1.201692, y=-0.495755, |F|=5.433e-02
iter 11: x=1.147771, y=-0.487332, |F|=5.578e-03
iter 12: x=1.142256, y=-0.486273, |F|=5.407e-05
iter 13: x=1.142203, y=-0.486262, |F|=5.018e-09
iter 14: x=1.142203, y=-0.486262, |F|=2.605e-17
x=1.142203, y=-0.486262, итераций=14, ошибка=2.6053355513225265e-17


In [7]:
import math

eps = 1e-15
maxIter = 50


def systemEquations(x, y):
    """
    Система уравнений:
    f1 = x^5 + y^4 - 2
    f2 = (x-2)^3 + (y-2)^3 + 16
    """
    f1 = x**5 + y**4 - 2.0
    f2 = (x - 2)**3 + (y - 2)**3 + 16.0
    return [f1, f2]


def systemJacobian(x, y):
    """
    Матрица Якоби:
    df1/dx = 5x^4
    df1/dy = 4y^3
    df2/dx = 3(x-2)^2
    df2/dy = 3(y-2)^2
    """
    j00 = 5 * x**4
    j01 = 4 * y**3
    j10 = 3 * (x - 2)**2
    j11 = 3 * (y - 2)**2
    return [j00, j01, j10, j11]


def computeAlphaCoefficients(j):
    """
    Весовые коэффициенты Федоренко
    """
    normF1 = math.sqrt(j[0]**2 + j[1]**2)
    normF2 = math.sqrt(j[2]**2 + j[3]**2)
    return [normF1, normF2]


def computeNorm(f, alpha):
    """
    Вычисление взвешенной нормы
    """
    term1 = (f[0] / alpha[0])**2 if alpha[0] != 0 else 0
    term2 = (f[1] / alpha[1])**2 if alpha[1] != 0 else 0
    return math.sqrt(term1 + term2)


def solve2x2System(j, f):
    """
    Решение системы J * dx = -F формулами Крамера
    """
    a, b, c, d = j
    det = a * d - b * c

    if abs(det) < 1e-30:
        raise ValueError("Якобиан вырожден")

    dx1 = ( (-f[0]) * d - b * (-f[1]) ) / det
    dx2 = ( a * (-f[1]) - (-f[0]) * c ) / det
    return [dx1, dx2]


def fedorenkoMethod(x0, y0, eps=eps, maxIter=maxIter, verbose=False):
    """
    Метод Ньютона с модификацией Федоренко
    """
    x, y = x0, y0
    oldNorm = 1e300

    for iteration in range(maxIter):
        f = systemEquations(x, y)
        j = systemJacobian(x, y)

        alpha = computeAlphaCoefficients(j)
        currentNorm = computeNorm(f, alpha)

        if verbose:
            print(f"iter {iteration:2d}: x={x:.6f}, y={y:.6f}, |F|={currentNorm:.3e}")

        if currentNorm < eps:
            return x, y, iteration, currentNorm

        dx = solve2x2System(j, f)

        tau = 1.0
        while tau > 1e-15:
            xNew = x + tau * dx[0]
            yNew = y + tau * dx[1]

            fNew = systemEquations(xNew, yNew)
            newNorm = computeNorm(fNew, alpha)

            if newNorm < currentNorm:
                x, y = xNew, yNew
                break

            tau /= 2

        if tau <= 1e-15:
            if verbose:
                print("Шаг не удалось уменьшить — остановка")
            break

    return x, y, maxIter, -1


result = fedorenkoMethod(2.0, 3.0, verbose=True)
x, y, iters, err = result
print(f"x={x:.6f}, y={y:.6f}, итераций={iters}, ошибка={err}")


iter  0: x=2.000000, y=3.000000, |F|=5.727e+00
iter  1: x=2.782812, y=2.291667, |F|=8.915e+00
iter  2: x=2.779014, y=2.315083, |F|=8.969e+00
iter  3: x=2.778542, y=2.317914, |F|=8.975e+00
iter  4: x=2.778439, y=2.318530, |F|=8.977e+00
iter  5: x=3.119913, y=0.277460, |F|=1.415e+00
iter  6: x=2.500342, y=-0.841726, |F|=5.672e-01
iter  7: x=2.011562, y=-0.544943, |F|=3.798e-01
iter  8: x=1.632799, y=-0.520081, |F|=2.723e-01
iter  9: x=1.360594, y=-0.511464, |F|=1.594e-01
iter 10: x=1.201692, y=-0.495755, |F|=5.433e-02
iter 11: x=1.147771, y=-0.487332, |F|=5.578e-03
iter 12: x=1.142256, y=-0.486273, |F|=5.407e-05
iter 13: x=1.142203, y=-0.486262, |F|=5.018e-09
iter 14: x=1.142203, y=-0.486262, |F|=2.605e-17
x=1.142203, y=-0.486262, итераций=14, ошибка=2.6053355513225265e-17
